In [ ]:
import warnings
from functools import partial
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import optuna
import polars as pl
import polars.selectors as cs

from catboost import CatBoostRegressor, MultiTargetCustomMetric
from xgboost import XGBRegressor

from numpy.typing import ArrayLike, NDArray
from polars.testing import assert_frame_equal
from sklearn.base import BaseEstimator
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings("ignore", message="Failed to optimize method")


#DATA_DIR = Path("/kaggle/input/child-mind-institute-problematic-internet-use")
DATA_DIR = Path("./data")

TARGET_COLS = [
    "PCIAT-PCIAT_01",
    "PCIAT-PCIAT_02",
    "PCIAT-PCIAT_03",
    "PCIAT-PCIAT_04",
    "PCIAT-PCIAT_05",
    "PCIAT-PCIAT_06",
    "PCIAT-PCIAT_07",
    "PCIAT-PCIAT_08",
    "PCIAT-PCIAT_09",
    "PCIAT-PCIAT_10",
    "PCIAT-PCIAT_11",
    "PCIAT-PCIAT_12",
    "PCIAT-PCIAT_13",
    "PCIAT-PCIAT_14",
    "PCIAT-PCIAT_15",
    "PCIAT-PCIAT_16",
    "PCIAT-PCIAT_17",
    "PCIAT-PCIAT_18",
    "PCIAT-PCIAT_19",
    "PCIAT-PCIAT_20",
    "PCIAT-PCIAT_Total",
    "sii",
]

FEATURE_COLS = [
    "Basic_Demos-Age",
    "PreInt_EduHx-computerinternet_hoursday",
    "SDS-SDS_Total_Raw",
    "Physical-Height",
    "FGC-FGC_TL",
    "Physical-Waist_Circumference",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "Fitness_Endurance-Time_Sec",
    "Fitness_Endurance-Max_Stage",
    "FGC-FGC_CU_Zone",
]

In [ ]:
# Load data
train = pl.read_csv(DATA_DIR / "train.csv")
test = pl.read_csv(DATA_DIR / "test.csv")
train_test = pl.concat([train, test], how="diagonal")

IS_TEST = test.height <= 100

assert_frame_equal(train, train_test[: train.height].select(train.columns))
assert_frame_equal(test, train_test[train.height :].select(test.columns))

In [ ]:
# Cast string columns to categorical
train_test = train_test.with_columns(cs.string().cast(pl.Categorical).fill_null("NAN"))
train = train_test[: train.height]
test = train_test[train.height :]

# ignore rows with null values in TARGET_COLS
train_without_null = train_test.drop_nulls(subset=TARGET_COLS)
X = train_without_null.select(FEATURE_COLS)
X_test = test.select(FEATURE_COLS)
y = train_without_null.select(TARGET_COLS)
y_sii = y.get_column("sii").to_numpy()  # ground truth
cat_features = X.select(cs.categorical()).columns

print("Features selected:")
print(cat_features)  # Should be none

# Tubotubo's Quadratic Weighted Kappa metric & Optuna optimizer

In [ ]:
class MultiTargetQWK(MultiTargetCustomMetric):
    def get_final_error(self, error, weight):
        return np.sum(error)  # / np.sum(weight)

    def is_max_optimal(self):
        # if True, the bigger the better
        return True

    def evaluate(self, approxes, targets, weight):
        # approxes: 予測値 (shape: [ターゲット数, サンプル数])
        # targets: 実際の値 (shape: [ターゲット数, サンプル数])
        # weight: サンプルごとの重み (Noneも可)

        approx = np.clip(approxes[-1], 0, 3).round().astype(int)
        target = targets[-1]

        qwk = cohen_kappa_score(target, approx, weights="quadratic")

        return qwk, 1

    def get_custom_metric_name(self):
        return "MultiTargetQWK"


class OptimizedRounder:
    """
    A class for optimizing the rounding of continuous predictions into discrete class labels using Optuna.
    The optimization process maximizes the Quadratic Weighted Kappa score by learning thresholds that separate
    continuous predictions into class intervals.

    Args:
        n_classes (int): The number of discrete class labels.
        n_trials (int, optional): The number of trials for the Optuna optimization. Defaults to 100.

    Attributes:
        n_classes (int): The number of discrete class labels.
        labels (NDArray[np.int_]): An array of class labels from 0 to `n_classes - 1`.
        n_trials (int): The number of optimization trials.
        metric (Callable): The Quadratic Weighted Kappa score metric used for optimization.
        thresholds (List[float]): The optimized thresholds learned after calling `fit()`.

    Methods:
        fit(y_pred: NDArray[np.float_], y_true: NDArray[np.int_]) -> None:
            Fits the rounding thresholds based on continuous predictions and ground truth labels.

            Args:
                y_pred (NDArray[np.float_]): Continuous predictions that need to be rounded.
                y_true (NDArray[np.int_]): Ground truth class labels.

            Returns:
                None

        predict(y_pred: NDArray[np.float_]) -> NDArray[np.int_]:
            Predicts discrete class labels by rounding continuous predictions using the fitted thresholds.
            `fit()` must be called before `predict()`.

            Args:
                y_pred (NDArray[np.float_]): Continuous predictions to be rounded.

            Returns:
                NDArray[np.int_]: Predicted class labels.

        _normalize(y: NDArray[np.float_]) -> NDArray[np.float_]:
            Normalizes the continuous values to the range [0, `n_classes - 1`].

            Args:
                y (NDArray[np.float_]): Continuous values to be normalized.

            Returns:
                NDArray[np.float_]: Normalized values.

    References:
        - This implementation uses Optuna for threshold optimization.
        - Quadratic Weighted Kappa is used as the evaluation metric.
    """

    def __init__(self, n_classes: int, n_trials: int = 100):
        self.n_classes = n_classes
        self.labels = np.arange(n_classes)
        self.n_trials = n_trials
        self.metric = partial(cohen_kappa_score, weights="quadratic")

    def fit(self, y_pred: NDArray[np.float_], y_true: NDArray[np.int_]) -> None:
        y_pred = self._normalize(y_pred)

        def objective(trial: optuna.Trial) -> float:
            thresholds = []
            for i in range(self.n_classes - 1):
                low = max(thresholds) if i > 0 else min(self.labels)
                high = max(self.labels)
                th = trial.suggest_float(f"threshold_{i}", low, high)
                thresholds.append(th)
            try:
                y_pred_rounded = np.digitize(y_pred, thresholds)
            except ValueError:
                return -100
            return self.metric(y_true, y_pred_rounded)

        optuna.logging.disable_default_handler()
        study = optuna.create_study(direction="maximize")
        study.optimize(
            objective,
            n_trials=self.n_trials,
        )
        self.thresholds = [study.best_params[f"threshold_{i}"] for i in range(self.n_classes - 1)]

    def predict(self, y_pred: NDArray[np.float_]) -> NDArray[np.int_]:
        assert hasattr(self, "thresholds"), "fit() must be called before predict()"
        y_pred = self._normalize(y_pred)
        return np.digitize(y_pred, self.thresholds)

    def _normalize(self, y: NDArray[np.float_]) -> NDArray[np.float_]:
        # normalize y_pred to [0, n_classes - 1]
        return (y - y.min()) / (y.max() - y.min()) * (self.n_classes - 1)

# Start making models!

In [ ]:
# setting catboost parameters
params = dict(
    loss_function="MultiRMSE",
    eval_metric=MultiTargetQWK(),
    iterations=1 if IS_TEST else 100000,
    learning_rate=0.1,
    depth=5,
    early_stopping_rounds=50,
)

# Cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=52)
models: list[CatBoostRegressor] = []
y_pred = np.full((X.height, len(TARGET_COLS)), fill_value=np.nan)
for train_idx, val_idx in skf.split(X, y_sii):
    X_train: pl.DataFrame
    X_val: pl.DataFrame
    y_train: pl.DataFrame
    y_val: pl.DataFrame
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # train model
    model = CatBoostRegressor(**params)
    model.fit(
        X_train.to_pandas(),
        y_train.to_pandas(),
        eval_set=(X_val.to_pandas(), y_val.to_pandas()),
        cat_features=cat_features,
        verbose=False,
    )
    models.append(model)

    # predict
    y_pred[val_idx] = model.predict(X_val.to_pandas())

assert np.isnan(y_pred).sum() == 0
# Optimize thresholds
optimizer = OptimizedRounder(n_classes=4, n_trials=300)
y_pred_total = y_pred[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
optimizer.fit(y_pred_total, y_sii)
y_pred_rounded = optimizer.predict(y_pred_total)

# Calculate QWK
qwk = cohen_kappa_score(y_sii, y_pred_rounded, weights="quadratic")
print(f"Cross-Validated QWK Score: {qwk}")

In [ ]:
print(models)

In [ ]:
# Let's compare the CatBoost models
for model in models:
    #print(X, y_sii)
    y_pred = model.predict(X.to_pandas())
    #print(X.shape)
    #print(y_sii.shape)
    #print(y_pred.shape)
    y_pred_total = y_pred[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
    #print(y_pred_total.shape)
    
    y_pred_rounded = optimizer.predict(y_pred_total)    
    qwk = cohen_kappa_score(y_sii, y_pred_rounded, weights="quadratic")
    print(f"Cross-Validated QWK Score: {qwk}")


# Trying out XGBoost

In [ ]:
# setting xgboost parameters
'''params = dict(
    loss_function="MultiRMSE",
    eval_metric=MultiTargetQWK(),
    iterations=1 if IS_TEST else 100000,
    learning_rate=0.1,
    depth=5,
    early_stopping_rounds=50,
)'''

params = dict(
    n_estimators=1000, 
    max_depth=7, 
    eta=0.1, 
    subsample=0.7, 
    colsample_bytree=0.8
)

# Cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=52)
models: list[XGBRegressor] = []
y_pred = np.full((X.height, len(TARGET_COLS)), fill_value=np.nan)
for train_idx, val_idx in skf.split(X, y_sii):
    
    X_train: pl.DataFrame
    X_val: pl.DataFrame
    y_train: pl.DataFrame
    y_val: pl.DataFrame
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # train model
    model = XGBRegressor(**params)
    #print(X_train)
    #print(y_train)
    model.fit(
        X_train.to_pandas(),
        y_train.to_pandas(),
        eval_set=[(X_val.to_pandas(), y_val.to_pandas())],
        verbose=False,
    )
    models.append(model)

    # predict
    y_pred[val_idx] = model.predict(X_val.to_pandas())

assert np.isnan(y_pred).sum() == 0
# Optimize thresholds
optimizer = OptimizedRounder(n_classes=4, n_trials=300)
y_pred_total = y_pred[:, TARGET_COLS.index("PCIAT-PCIAT_Total")]
optimizer.fit(y_pred_total, y_sii)
y_pred_rounded = optimizer.predict(y_pred_total)

# Calculate QWK
qwk = cohen_kappa_score(y_sii, y_pred_rounded, weights="quadratic")
print(f"Cross-Validated QWK Score: {qwk}")

# Hyperparameter tuning

Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, cohen_kappa_score
import numpy as np

# Custom scorer for QWK
def qwk_scorer(y_true, y_pred):
    y_pred_rounded = np.rint(y_pred).astype(int)
    return cohen_kappa_score(y_true, y_pred_rounded, weights="quadratic")

qwk_scorer = make_scorer(qwk_scorer, greater_is_better=True)

# Define parameter grid
param_grid = {
    'max_depth': [5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [500, 1000],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.7, 0.9],
}

# Initialize the model
model = XGBRegressor()

# Initialize Grid Search
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=qwk_scorer,
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Fit Grid Search
grid_search.fit(X.to_pandas(), y_sii)

# Best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best QWK score:", grid_search.best_score_)


Random Search lol

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define parameter distributions
param_dist = {
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.2, 20),
    'n_estimators': np.arange(100, 1100, 100),
    'subsample': np.linspace(0.5, 1.0, 6),
    'colsample_bytree': np.linspace(0.5, 1.0, 6),
}

# Initialize Random Search
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter settings that are sampled
    scoring=qwk_scorer,
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit Random Search
random_search.fit(X.to_pandas(), y_sii)

# Best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best QWK score:", random_search.best_score_)

Hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(params):
    model = XGBRegressor(**params)
    score = cross_val_score(model, X.to_pandas(), y_sii, cv=5, scoring=qwk_scorer, n_jobs=-1).mean()
    return {'loss': -score, 'status': STATUS_OK}

# Define the search space
space = {
    'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1100, 100, dtype=int)),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

# Run optimization
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.RandomState(42)
)

# Best parameters
print("Best parameters:", best)


Bayesian Optimization with Optuna

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
    }
    model = XGBRegressor(**params)
    score = cross_val_score(model, X.to_pandas(), y_sii, cv=5, scoring=qwk_scorer, n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)
print("Best QWK score:", study.best_value)


# Optimizing CatBoost

In [ ]:
# setting catboost parameters
params = dict(
    loss_function="MultiRMSE",
    eval_metric=MultiTargetQWK(),
    iterations=1 if IS_TEST else 100000,
    learning_rate=0.1,
    depth=5,
    early_stopping_rounds=50,
)
o_params = {
    "n_classes": 4,
    "n_trials": 300
}
    
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, cohen_kappa_score


def objective(trial):
    model_params = {
        'loss_function': "MultiRMSE",
        'eval_metric': MultiTargetQWK(),
        'iterations': 10000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 3, 10, step=1),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 40, 60, step=10),
    }
    optimizer_params = {
        "n_classes": trial.suggest_int("n_classes", 3, 7, step=1),
        "n_trials": trial.suggest_int("n_trials", 100, 400, step=50),
    }
    
    model = CatBoostRegressor(**model_params)
    
    
    def qwk_scorer(y_true, y_pred):
        optimizer = OptimizedRounder(**optimizer_params)
        optimizer.fit(y_pred, y_true)
        y_pred_rounded = optimizer.predict(y_pred_total)
        
        #y_pred_rounded = np.rint(y_pred).astype(int)
        return cohen_kappa_score(y_true, y_pred_rounded, weights="quadratic")

    qwk_scorer = make_scorer(qwk_scorer, greater_is_better=True)
    
    score = cross_val_score(model, X.to_pandas(), y_sii, cv=5, scoring=qwk_scorer, n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)
print("Best QWK score:", study.best_value)

In [ ]:
# setting catboost parameters
params = dict(
    loss_function="MultiRMSE",
    eval_metric=MultiTargetQWK(),
    iterations=1 if IS_TEST else 100000,
    learning_rate=0.1,
    depth=5,
    early_stopping_rounds=50,
)
o_params = {
    "n_classes": 4,
    "n_trials": 300
}
    
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, cohen_kappa_score


def objective(trial):
    model_params = {
        'loss_function': "MultiRMSE",
        'eval_metric': MultiTargetQWK(),
        'iterations': 10000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 3, 10, step=1),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 40, 60, step=10),
    }
    '''optimizer_params = {
        "n_classes": trial.suggest_int("n_classes", 3, 7, step=1),
        "n_trials": trial.suggest_int("n_trials", 100, 400, step=50),
    }'''
    
    model = CatBoostRegressor(**model_params)
    
    def qwk_scorer(y_true, y_pred):
        #optimizer = OptimizedRounder(**optimizer_params)
        #optimizer.fit(y_pred, y_true)
        #y_pred_rounded = optimizer.predict(y_pred_total)
        #y_pred_rounded = np.rint(y_pred).astype(int)
        return cohen_kappa_score(y_true, y_pred, weights="quadratic")

    qwk_scorer = make_scorer(qwk_scorer, greater_is_better=True)
    
    score = cross_val_score(model, X.to_pandas(), y_sii, cv=5, scoring=qwk_scorer, n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best parameters:", study.best_params)
print("Best QWK score:", study.best_value)